# TC-reporting tool

If necessary, only change the capitalized variables.

In [1]:
from TC_helper_functions import *

### I. LOAD THE DATA & MODEL

In [2]:
# IMPORT THE JIRA DATA
DATA_INPUT_DIR = "data/03_input"
input_dataset = load_dataset_improved(DATA_INPUT_DIR)

# LOAD THE MODEL (Currently trained on 1.2019-8.2021 data)
MODEL_DIR = "model/TC_model.p"
model_dictized = pickle.load(open(MODEL_DIR, "rb"))

database = model_dictized["model_database"]
vectorizer = model_dictized["model_vectorizer"]
doc_term_matrix = model_dictized["model_docterm"]

Number of tickets before any filtering: 82.


### II. REQUIRED FILTERING FOR THE DATA SET

In [3]:
# NON-EVENT TICKETS
df = input_dataset[input_dataset['Labels'] != "event"].copy()
print("Number of non-event tickets: {}.".format(len(df)))

# SELECT ISSUE TYPE
ISSUE_TYPE = "Service Request"

print("Possible types of issues: {}.".format(set(df["Issue Type"])))
df = df[df["Issue Type"] == ISSUE_TYPE]
print("\nNumber of {} type of issues: {}.".format(ISSUE_TYPE, len(df)))

# REARRANGE
df = df[['Reporter','Assignee', "Custom field (Customer)", "Summary", "Description", "Created", "Resolved", "Issue key"]]

# DROP CRITC. NAN
df = df.dropna(subset=['Summary', 'Description', 'Created'])
print("Number of tickets after removing samples with nan: {}.".format(len(df)))

df = df.reset_index(drop=True)

# DEFINE & LOAD STOPWORDS
DATA_STOPWORDS_DIR = "data/02_auxiliary"

stopwords_set = load_stopwords(DATA_STOPWORDS_DIR, general=True, it=True, names=True)
print("Number of stopwords loaded: {}.".format(len(stopwords_set)))

# RUN NLP-process
dataset = df.copy()
#dataset = df_EFIMA.copy()

dataset["ProcessedWords"] = dataset.apply(lambda x: remove_stopwords(get_nlp_processed_text(x.Summary, x.Description), stopwords_set), axis=1)
dataset["ProcessedSentence"] =  dataset.apply(lambda x: ' '.join(x.ProcessedWords), axis=1)

Number of non-event tickets: 69.
Possible types of issues: {'Service Request', 'Incident', 'Access'}.

Number of Service Request type of issues: 49.
Number of tickets after removing samples with nan: 49.
Number of stopwords loaded: 41100.


### III. BUILD THE OUTPUT MATRIX FOR RECOMMENDATIONS

In [4]:
N = 4
PREFIX = "https://support.efima.com/jira/issues/?jql=issuekey%20in%20("

EXTENSION = get_inference_extension(dataset, PREFIX, N, vectorizer, doc_term_matrix, database)
ARRIVED_TICKET_PREFIX = "https://support.efima.com/jira/issues/?jql=issuekey="

OUTPUT_DATAFRAME = df[["Issue key"]].rename(columns={"Issue key": "Unresolved"})
OUTPUT_DATAFRAME["URL"] = OUTPUT_DATAFRAME.apply(lambda x: ARRIVED_TICKET_PREFIX + x.Unresolved, axis=1)

OUTPUT = pd.concat([OUTPUT_DATAFRAME, EXTENSION], axis=1)

### IV. SAVE THE OUTPUT INTO .CSV FILE

In [9]:
import datetime
  
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
#path = "outputs/" + timestamp + "_TC_recommendations.csv" 
OUTPUT.to_csv("outputs/TC_" + str(timestamp) + ".csv", index=False)

In [8]:
OUTPUT

,Unresolved,URL,Minimum,Maximum,Recommendations-URL
0,ESD-206803,https://support.efima.com/jira/issues/?jql=iss...,1.0,1.0,https://support.efima.com/jira/issues/?jql=iss...
1,ESD-206746,https://support.efima.com/jira/issues/?jql=iss...,0.47,0.57,https://support.efima.com/jira/issues/?jql=iss...
2,ESD-206741,https://support.efima.com/jira/issues/?jql=iss...,0.33,0.38,https://support.efima.com/jira/issues/?jql=iss...
3,ESD-206710,https://support.efima.com/jira/issues/?jql=iss...,0.32,0.48,https://support.efima.com/jira/issues/?jql=iss...
4,ESD-206638,https://support.efima.com/jira/issues/?jql=iss...,0.73,0.78,https://support.efima.com/jira/issues/?jql=iss...
5,ESD-206600,https://support.efima.com/jira/issues/?jql=iss...,0.82,0.86,https://support.efima.com/jira/issues/?jql=iss...
6,ESD-206531,https://support.efima.com/jira/issues/?jql=iss...,0.75,0.77,https://support.efima.com/jira/issues/?jql=iss...
7,ESD-206482,https://support.efima.com/jira/issues/?jql=iss...,0.4,0.55,https://support.efima.com/jira/issues/?jql=iss...
8,ESD-206456,https://support.efima.com/jira/issues/?jql=iss...,0.28,0.49,https://support.efima.com/jira/issues/?jql=iss...
9,ESD-206455,https://support.efima.com/jira/issues/?jql=iss...,0.35,0.51,https://support.efima.com/jira/issues/?jql=iss...
